# Kapitel 4: Die Schnellstraße ins Web – Der `requests`-Client

Wir haben im ersten Block gesehen, wie viel Handarbeit nötig ist, um mit Sockets eine einfache Web-Anfrage zu stellen. Wir mussten Verbindungen manuell aufbauen, HTTP-Requests als String formulieren, auf korrekte Zeilenumbrüche achten, Daten in Bytes umwandeln und die Verbindung wieder schließen. Das ist umständlich und fehleranfällig. In diesem Kapitel lernen wir die Bibliothek kennen, die all das für uns erledigt: `requests`.

## Vom Handwerk zur Ingenieurskunst: Effiziente API-Kommunikation

### 4.1 Vorstellung von `requests`: Warum es der De-facto-Standard ist

`requests` ist eine Drittanbieter-Bibliothek für Python, die nicht zur Standardbibliothek gehört. Sie ist jedoch so populär und weit verbreitet, dass sie als der Industriestandard für HTTP-Kommunikation in Python gilt.

**Warum ist `requests` so viel besser als die manuelle Socket-Programmierung?**

  * **Einfachheit:** Eine einzige Funktion (`requests.get()`) erledigt die Arbeit von dutzenden Zeilen Socket-Code.
  * **Automatismen:** `requests` kümmert sich automatisch um:
      * Connection-Management und Wiederverwendung (Keep-Alive).
      * Automatische Verfolgung von Weiterleitungen (Redirects, z.B. von HTTP zu HTTPS).
      * Cookie-Handhabung.
      * Dekodierung des Inhalts.
  * **Daten-Handling:** JSON-Antworten können mit einer einzigen Methode (`.json()`) direkt in Python-Dictionaries umgewandelt werden.

**Installation:**
Da `requests` keine Standardbibliothek ist, muss sie zuerst über den Paketmanager `pip` installiert werden. Öffnen Sie Ihr Terminal und führen Sie aus:

```bash
pip install requests
```

### 4.2 Die `GET`-Anfrage und das `Response`-Objekt

Die mit Abstand häufigste Interaktion mit einer API ist das Abrufen von Daten. Dies geschieht mit einer `GET`-Anfrage. Mit `requests` ist das denkbar einfach:
`response = requests.get('https://api.beispiel.de/daten')`

Das wirklich Mächtige ist das Objekt, das dieser Aufruf zurückgibt: ein **`Response`-Objekt**. Dieses Objekt ist ein Container, der alles enthält, was der Server uns als Antwort geschickt hat.

**Die wichtigsten Attribute des `Response`-Objekts:**

| Attribut / Methode | Beschreibung | Beispiel-Wert |
| :--- | :--- | :--- |
| `response.status_code`| Der 3-stellige HTTP-Status-Code als Integer. **Immer prüfen\!** | `200`, `404`, `500` |
| `response.ok` | Ein Boolean, der `True` ist, wenn der Status-Code unter 400 liegt (also kein Client- oder Serverfehler). | `True`, `False` |
| `response.text` | Der Inhalt (Body) der Antwort als String. Ideal für HTML oder reinen Text. | `"<html>...</html>"` |
| `response.content` | Der Inhalt (Body) der Antwort als rohe Bytes. Nötig für Bilder oder andere Nicht-Text-Dateien. | `b'\x89PNG...'` |
| `response.json()` | **Methode**, die versucht, den Body als JSON zu deserialisieren und als Python-`dict` oder `list` zurückzugeben. | `{'id': 1, 'name': 'Max'}` |
| `response.headers` | Ein Dictionary-ähnliches Objekt, das alle Antwort-Header enthält. | `{'Content-Type': 'application/json', ...}`|
| `response.url` | Die finale URL der Anfrage, nachdem allen Weiterleitungen gefolgt wurde. | `"https://www.google.com/"` |

### 4.3 Anfragen filtern mit Query-Parametern

Viele APIs erlauben es, die Ergebnisse durch Parameter zu filtern, die an die URL angehängt werden (z.B. `?user=123&sort=desc`). Diese von Hand in die URL einzubauen ist fehleranfällig (z.B. wegen Sonderzeichen). `requests` bietet hierfür das `params`-Argument. Man übergibt ein Python-Dictionary, und `requests` formatiert es korrekt und hängt es an die URL an.

-----

## Die `requests`-Bibliothek in Aktion

### Beispiel 1: Eine einfache GET-Anfrage und Status-Prüfung

Wir senden eine einfache Anfrage und prüfen, ob sie erfolgreich war. Das ist der grundlegendste Schritt bei jeder API-Kommunikation.



```python
import requests

# Die URL des API-Endpunkts, den wir anfragen wollen
url = "https://api.github.com"

# Sende die GET-Anfrage
response = requests.get(url)

# Überprüfe den Status-Code der Antwort
if response.status_code == 200:
    # 200 bedeutet "OK" - die Anfrage war erfolgreich
    print("Anfrage erfolgreich!")
    print(f"Status-Code: {response.status_code}")
else:
    # Jeder andere Code deutet auf ein Problem hin
    print("Anfrage fehlgeschlagen.")
    print(f"Status-Code: {response.status_code}")

# Man kann auch das .ok-Attribut für eine schnelle Prüfung verwenden
if response.ok:
    print("Die Antwort ist OK (Status < 400).")
```



### Beispiel 2: Den Inhalt der Antwort inspizieren

Jetzt schauen wir uns an, was der Server uns geschickt hat. Wir nutzen die JSONPlaceholder-API, die uns Test-Daten im JSON-Format liefert.

```python
import requests

# Wir fragen ein einzelnes "To-Do"-Objekt an
url = "https://jsonplaceholder.typicode.com/todos/1"

response = requests.get(url)

if response.ok:
    # 1. Die Antwort als reinen Text ausgeben
    print("--- Antwort als Text ---")
    print(response.text)
    
    # 2. Die Antwort mit .json() direkt in ein Python-Dictionary umwandeln
    # Dies ist der bevorzugte Weg für JSON-APIs.
    data = response.json()
    
    print("\n--- Antwort als Python-Dictionary ---")
    print(data)
    print(f"Typ der Daten: {type(data)}")
    
    # 3. Auf die Daten wie in einem normalen Dictionary zugreifen
    print(f"\nDer Titel des To-Do's ist: '{data['title']}'")
    
```

### Beispiel 3: Query-Parameter verwenden

Wir wollen nicht alle Kommentare von JSONPlaceholder, sondern nur die, die zu einem bestimmten Post gehören. Dazu filtern wir die Anfrage mit dem `params`-Argument.

```python
import requests

# Der Basis-Endpunkt für Kommentare
url = "https://jsonplaceholder.typicode.com/comments"

# Ein Dictionary, das unsere Filterkriterien enthält.
# Wir wollen alle Kommentare, bei denen das Feld 'postId' den Wert 1 hat.
query_params = {
    "postId": 1
}

# Übergib das Dictionary an das 'params'-Argument.
response = requests.get(url, params=query_params)

if response.ok:
    # Zeige die URL an, die requests tatsächlich aufgerufen hat.
    # Du siehst, dass ?postId=1 angehängt wurde.
    print(f"Aufgerufene URL: {response.url}")
    
    # Die Antwort ist eine Liste von Kommentaren
    comments = response.json()
    print(f"\nAnzahl der Kommentare für Post 1 gefunden: {len(comments)}")
    
    # Gib die E-Mail des ersten Kommentators aus
    if comments:
        print(f"E-Mail des ersten Kommentators: {comments[0]['email']}")

```

-----

## Das Projekt wird zum Client: Bücher von der API abrufen

Jetzt nutzen wir `requests`, um unser Tutor-Projekt voranzubringen. Wir rufen gezielt Daten für ein bestimmtes Buch von der Open Library API ab. Diese API benötigt mehrere Parameter, was sie zu einem perfekten Beispiel für das `params`-Argument macht.

```python
import requests
import json

# Basis-URL der API
base_url = "https://openlibrary.org/api/books"

# Parameter für unsere Anfrage definieren
# Wir fragen nach einem Buch über seine ISBN.
# Wir wollen die Antwort im JSON-Format.
# 'jscmd=data' ist eine API-spezifische Anforderung für Detailinformationen.
params = {
    "bibkeys": "ISBN:9780132350884", # ISBN für "Clean Code"
    "format": "json",
    "jscmd": "data"
}

print(f"Frage Buchdaten für {params['bibkeys']} an...")

# Sende die GET-Anfrage
response = requests.get(base_url, params=params)

# Prüfe, ob die Anfrage erfolgreich war
if response.ok:
    # Wandle die JSON-Antwort in ein Python-Dictionary um
    book_data = response.json()
    
    # Die Struktur dieser API ist etwas speziell:
    # Die Antwort ist ein Dictionary, bei dem unser angefragter "bibkey" der Schlüssel ist.
    book_key = params["bibkeys"]
    
    if book_key in book_data:
        # Extrahiere die gewünschten Informationen
        title = book_data[book_key]["title"]
        # Autoren ist eine Liste, wir nehmen den Namen des ersten Autors
        author = book_data[book_key]["authors"][0]["name"]
        
        print("\n--- Buch gefunden! ---")
        print(f"Titel: {title}")
        print(f"Autor: {author}")
    else:
        print("Fehler: Das Buch konnte in der Antwort nicht gefunden werden.")

else:
    print(f"Fehler bei der Anfrage. Status-Code: {response.status_code}")
```

-----

### Deine Werkstatt: Übungen zu Kapitel 4

# Übungs-Set 4: Daten mit `requests` abrufen

**Ziel:** In diesen Übungen werden Sie sicher im Umgang mit der `requests`-Bibliothek, um Daten von einer Live-API abzurufen und zu verarbeiten.

### Aufgabe 1: Einfacher API-Ping

Senden Sie eine `GET`-Anfrage an `https://jsonplaceholder.typicode.com/posts`. Geben Sie nur aus, ob die Anfrage erfolgreich war (`response.ok`) und welchen Status-Code Sie erhalten haben.

### Aufgabe 2: Einen einzelnen Datensatz abrufen

Rufen Sie von JSONPlaceholder den einzelnen Post mit der `id` 42 ab (Endpunkt: `/posts/42`). Wandeln Sie die Antwort in ein Python-Dictionary um und geben Sie den `title` dieses Posts aus.

### Aufgabe 3: Eine Liste von Daten verarbeiten

Rufen Sie die Liste aller Benutzer vom Endpunkt `/users` ab. Die Antwort wird eine Liste von Dictionaries sein. Schreiben Sie eine Schleife, die über diese Liste iteriert und für jeden Benutzer seinen `name` und seine `email` ausgibt.

### Aufgabe 4: **Schüler-Projekt: Posts eines Benutzers filtern**

Schreiben Sie ein Skript, das alle Posts des Benutzers mit der `userId` 5 von JSONPlaceholder abruft.

  * Der Endpunkt ist `/posts`.
  * Nutzen Sie das `params`-Argument, um die Ergebnisse zu filtern.
  * Iterieren Sie durch die Antwort und geben Sie den Titel jedes Posts aus.

### Aufgabe 5: **Schüler-Projekt: Verknüpfte Daten abrufen**

Schreiben Sie ein Skript, das in zwei Schritten vorgeht:

1.  Rufen Sie zuerst den einzelnen Post mit der `id` 15 ab.
2.  Rufen Sie danach **alle Kommentare** für diesen Post ab. Der Endpunkt dafür ist `/posts/15/comments`.
    Geben Sie am Ende aus, wie viele Kommentare für Post 15 gefunden wurden.

### Aufgabe 6 (Challenge): Eine andere API nutzen - Pokémon\!

Nutzen Sie die öffentliche Pokémon-API.

1.  Schreiben Sie ein Skript, das den Benutzer nach dem Namen eines Pokémon fragt (z.B. "pikachu" oder "ditto").
2.  Bauen Sie die URL für den API-Endpunkt dynamisch zusammen: `https://pokeapi.co/api/v2/pokemon/{name_des_pokemons}`.
3.  Rufen Sie die Daten ab.
4.  Geben Sie den Namen des Pokémon, seine Höhe (`height`) und sein Gewicht (`weight`) aus dem JSON-Ergebnis aus.

### Aufgabe 7 (Challenge): Header inspizieren

Senden Sie eine `GET`-Anfrage an eine beliebige URL (z.B. `https://www.python.org`). Greifen Sie auf die `response.headers` zu und geben Sie die Werte für die Header `Content-Type` und `Server` aus.